#Exercise 4

For this exercise, we simulate Conway's Game of Life and compute entropy.

We use the following packages:

NumPy -> allows us to use some math operations and NumPy arrays

Plotly.express -> allows us to create plots in a single line

SciPy -> allows for easy calculation of the number of alive neighbours for each cell

Pandas -> for dataframes

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.ndimage import convolve
import pandas as pd

In [ ]:
# The board for Conway's game of life with each cell having value 0 or 1 and 50 rows and 100 columns.
x = np.random.randint(0,2,size=(50,100))

# Create figure of board and display it.
fig = px.imshow(x)
fig.show()


In [ ]:
# Update the board by one step.
def step(x):
    # The cells with 1 are the ones being considered for calculation
    kernel = np.array([[1,1,1], [1,0,1], [1,1,1]])
    # This determines the number of cells surrounding the center cell that contain a 1 for all cells in x
    # neighbours[2][4] = n, where n is the number of 1s surrounding the cell (2, 4)
    neighbours = convolve(x, kernel)
    # To store the changes
    diff = np.zeros_like(neighbours)
    # The following logic is cool!
    # This goes through neighbours and x (both 2d arrays) and wherever both conditions are true diff is modified at that cell with the indicated value
    diff[(neighbours < 2) & (x == 1)] = -1 # dies of loneliness
    diff[(neighbours > 3) & (x == 1)] = -1 # dies due to overcrowding
    diff[(neighbours == 3) & (x == 0)] = +1 # becomes alive
    return x + diff

l = []
for _ in range(50):
    # Append to the list l the board x. So l will contain all board for all time steps.
    l += [x]
    x = step(x)

fig = px.imshow(np.array(l), animation_frame=0)
fig.show()

In [ ]:
# The list l contains a new np.array for each time-step of the game of life
# Calculate, how the entropy changes over time and visualize the output
# We recommend to use a plotly line-chart for visualization https://plotly.com/python/line-charts/

#Solution

In [ ]:
def calc_entropy(probs):
    H = 0
    for p in probs:
        # The entropy equation from class (using change of base to get log base 2).
        if p > 0:
            H -= p * (np.log(p)/np.log(2))
    return H

cols = 100
rows = 50
x = np.random.randint(0,2,size=(rows,cols))

# Patterns as small numpy arrays containing 0/1.
BLOCK = [
    [1,1],
    [1,1]
]

BLINKER_H = [
    [1,1,1]
]

BLINKER_V = [
    [1],
    [1],
    [1]
]

GLIDER_1 = [
    [0,1,0],
    [0,0,1],
    [1,1,1]
]

patterns = {
    "block": BLOCK,
    "blinker_h": BLINKER_H,
    "blinker_v": BLINKER_V,
    "glider": GLIDER_1
}

def count_pattern(grid, pattern):
    grid = np.array(grid)
    ph, pw = pattern.shape
    gh, gw = grid.shape

    count = 0
    for i in range(gh - ph + 1):
        for j in range(gw - pw + 1):
            if np.array_equal(grid[i:i+ph, j:j+pw], pattern):
                count += 1
    return count

def get_pattern_histogram(grid, patterns):
    hist = {}
    for name, pat in patterns.items():
        hist[name] = count_pattern(grid, np.array(pat))
    return hist

def histogram_to_probabilities(hist):
    total = sum(hist.values())
    if total == 0:
        return {k: 0 for k in hist}  # avoid divide-by-zero
    return [hist[k] / total for k in hist]

l = []
row_probs = []
col_probs = []
pattern_probs = []
for idx in range(50):
    l += [x]
    # Get row and col entropy probabilities:
    cnt_row = np.array([sum(r) for r in l[idx]])
    cnt_col = np.array([sum(row[c] for row in l[idx]) for c in range(cols)])
    # Recall that the sum of probabilities needs to be 1.
    total = np.sum(l[idx])
    row_probs += [cnt_row / total]
    col_probs += [cnt_col / total]

    # Get pattern probabilities:
    pattern_hist = get_pattern_histogram(l[idx], patterns)
    pattern_probs += [histogram_to_probabilities(pattern_hist)]
    x = step(x)

row_entropys = [calc_entropy(p) for p in row_probs]
col_entropys = [calc_entropy(p) for p in col_probs]
pattern_entropys = [calc_entropy(p) for p in pattern_probs]

xs = [i for i in range(50)]
df = pd.DataFrame({
    "x": xs,
    "Row Entropy": row_entropys,
    "Column Entropy": col_entropys,
    "Pattern Entropy": pattern_entropys
})

fig = px.imshow(np.array(l), animation_frame=0)
fig.show()

fig = px.line(df, x="x", y=["Row Entropy", "Column Entropy", "Pattern Entropy"])
fig.show()